# VGG-16 as feature extractor

In [ ]:
from keras.preprocessing import image
from keras.optimizers import SGD
import os, sys
import numpy as np


from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

img_width, img_height = 224, 224
nb_epoch = 50
nb_train_samples = 5312
nb_validation_samples = 1332

model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2,2), strides=(2,2)))

model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2,2), strides=(2,2)))

model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2,2), strides=(2,2)))

model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3')
model_vgg.add(MaxPooling2D((2,2), strides=(2,2)))

model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1,1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2,2), strides=(2,2)))

model_vgg.add(Flatten(name="flatten"))
model_vgg.add(Dense(4096, activation='relu', name='dense_1'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(4096, activation='relu', name='dense_2'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(2, name='dense_3'))
model_vgg.add(Activation("softmax",name="softmax"))

model_vgg.load_weights('models/vgg16/final-model.h5', by_name=True)



In [ ]:
from keras.models import Model
vgg16_convolutional_only = Model(input=model_vgg.input, output=model_vgg.get_layer('dense_2').output) # TRUNCATE MODEL AT DENSE LAYER   



# Extract Train Data

In [ ]:
train_dat=np.empty((0,4096))
im_path = "DATA/Train/Others/"
dirs = os.listdir( im_path )
for item in dirs:
        if os.path.isfile(im_path+item):
            im = image.load_img(im_path+item,target_size=(224,224))
            im=image.img_to_array(im)
            im=np.expand_dims(im,axis=0)
            f=vgg16_convolutional_only.predict(im)
            train_dat=np.append(train_dat,f,axis=0)

In [ ]:
im_path = "DATA/Train/Vehicle/"
dirs = os.listdir( im_path )
for item in dirs:
        if os.path.isfile(im_path+item):
            im = image.load_img(im_path+item,target_size=(224,224))
            im=image.img_to_array(im)
            im=np.expand_dims(im,axis=0)
            f=vgg16_convolutional_only.predict(im)
            train_dat=np.append(train_dat,f,axis=0)

In [ ]:
y=np.array([0]*2656+ [1]*2656)
train_data=np.c_[y,train_dat]
np.savetxt('tr_dat.csv',train_data,delimiter=',')

# Extract Test Data

In [ ]:
test_dat=np.empty((0,4096))
im_path = "DATA/Test/Others/"
dirs = os.listdir( im_path )
for item in dirs:
        if os.path.isfile(im_path+item):
            im = image.load_img(im_path+item,target_size=(224,224))
            im=image.img_to_array(im)
            im=np.expand_dims(im,axis=0)
            f=vgg16_convolutional_only.predict(im)
            test_dat=np.append(test_dat,f,axis=0)

In [ ]:
im_path = "DATA/Test/Vehicle/"
dirs = os.listdir( im_path )
for item in dirs:
        if os.path.isfile(im_path+item):
            im = image.load_img(im_path+item,target_size=(224,224))
            im=image.img_to_array(im)
            im=np.expand_dims(im,axis=0)
            f=vgg16_convolutional_only.predict(im)
            test_dat=np.append(test_dat,f,axis=0)

In [ ]:
y=np.array([0]*666+ [1]*666)
test_data=np.c_[y,test_dat]
np.savetxt('te_dat.csv',test_data,delimiter=',')